---
layout: post
title: Blog Post 3
---

In this blog post, we’ll learn how to create a simple webapp using Flask. 

**I built my `app` file based on the example apps from the lecture and discussion, it is noted.**

# §1. What are we going to do?

We want to create a webapp including 3 pages:

1. **Main page**
2. **Submit a message**
3. **View messages**

For these pages, we need to create 3 templates seperately, which are `main.html`, `submit.html`, `view.html`.

Also, these 3 templates are supposed to be including in `base.html`.

# §2. Setup

## Base page

### - `base.html`

It can handle all the pages we will see on our website, which are `submit` page and `view` page. The reason why we do not contain our `main` page in it is that we only need our main(welcome) page to appear in the very beginning.

```html
{% raw %}
<!doctype html>
<title>{% block title %}{% endblock %} - Linqiao's Message Bank</title>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
<nav>
  <h1>Linqiao's Message Bank</h1>
  <!-- <b>Navigation:</b> -->
  <ul>
    <li><a href="{{ url_for('submit') }}">Submit a message</a></li>
    <li><a href="{{ url_for('view') }}">View messages</a></li>
  </ul>
</nav>
<section class="content">
  <header>
    {% block header %}{% endblock %}
  </header>
  {% block content %}{% endblock %}
</section>
{% endraw %}
```

## Main page

### - `main.html`

```html
{% raw %}
{% extends 'base.html' %}

{% block header %}
  <h1>{% block title %}Welcome{% endblock %}</h1>
{% endblock %}

{% block content %}
  <p>Welcome to Linqiao's Message Board!</p>
{% endblock %}
{% endraw %}
```

Create a WELCOME page by rendering `main.html` template.

In [ ]:
@app.route('/')
def main():

    return render_template('main.html')

![blog3_main.png](/images/blog3_main.png)

## Write two functions for database management

First, we should generate a new file called `app.py`, and we can write our functions in this file.

### - `get_message_db() `

This function should handle creating the database of messages.

In [ ]:
def get_message_db():
    
    # Check if there is our database in the g attribute of the app, 
    # and connect to that database if not
    if 'message_db' not in g:
        g.message_db = sqlite3.connect('messages_db.sqlite')
    
    cursor = g.message_db.cursor()
    
    # Check whether a table called messages exists in message_db, 
    # and create it if not. 
    # messages table have three columns, which are id, handle, and message.
    cursor.execute('CREATE TABLE IF NOT EXISTS messages(id INT, handle TEXT, message TEXT);')

    return g.message_db

### - `insert_message(request)` 

This function should insert input messages into the database of messages.

In [ ]:
def insert_message(request):
    
    # Reconnect to the database
    db = get_message_db()
    cursor = db.cursor()
    
    # Extract the message and the handle column from the database
    message = request.form["message"]
    handle = request.form["handle"]
    
    # Insert one user's message into the database
    row_num = cursor.execute("SELECT COUNT(*) FROM messages;").fetchone()[0]
    cursor.execute(f"""INSERT INTO messages (id, handle, message) VALUES ({row_num + 1}, "{handle}", "{message}");""")
    
    # db.commit() ensure our row insertion has been saved.
    db.commit()
    
    # close the database
    db.close()

## Submit page

### - `submit.html`

There are three user interface elements in `submit.html` template:

1. A submitting message box
2. A submitting user's name box
3. A “submit” button

If the text format is right, then print a message thanking the submission. 

If wrong, printing a error message telling them to input a vaild message again.

```html
{% raw %}
{% extends 'base.html' %}

{% block header %}
  <h1>{% block title %}Leave your message{% endblock %}</h1>
{% endblock %}

{% block content %}
  <form method="post" enctype="multipart/form-data">
      <label for="handle">What's your name?</label>
      <br>
      <input name="handle" id="handle">
      <br><br>
      <label for="message">What message do you have for Linqiao?</label>
      <br>
      <input type="text" name="message" id="message">
      <br><br>
      <input type="submit" value="Submit message">
  </form>

  {% if thanks %}
    <br>
    Thanks for posting your message!
  {% endif %}

  {% if error %}
    <br>
    Sorry, we couldn't read it. Please submit a vaild input.
  {% endif %}
{% endblock %}
{% endraw %}
```

Create a submit page by rendering `submit.html` template.

If in the **GET** case, just returning the template.

If in the **POST** case, then we will insert the message to hte database and response the user according to the validity of their input.

In [ ]:
@app.route('/submit/', methods=['POST', 'GET'])
def submit():

    if request.method == 'GET':
        return render_template('submit.html')
    else:
        try:
            insert_message(request)
            return render_template('submit.html', thanks = True)
        except:
            return render_template('submit.html', error = True)

![blog3_submit.png](/images/blog3_submit.png)

## View page

### - `view.html`

View some users' messages in this page.

```html
{% raw %}
{% extends 'base.html' %}

{% block header %}
  <h1>{% block title %}Message board{% endblock %}</h1>
{% endblock %}

{% block content %}
<ul>
  {% for m in messages %}
    <br><br> 
      <li>{{m[1]}}</li>
      <br> 
      <i>--- {{m[0]}}</i>
  {% endfor %}
</ul>
  
{% endblock %}
{% endraw %}
```

## Viewing Random Messages

### - `random_messages(n)` 

This function should return n random messages from the message_db. 

In [ ]:
def random_messages(n):
    
    # Reconnect to the database
    db = get_message_db()
    cursor = db.cursor()

    row_num = cursor.execute("SELECT COUNT(*) FROM messages;").fetchone()[0]
    
    # if the input number is more than the number of rows of data stored in the database
    # we will print all the data in the database
    if n > row_num:
        n = row_num
    
    # Randomly pick n messages corresponding to the user's handle from the database
    ran_messages = cursor.execute(f"""SELECT handle, message FROM messages ORDER BY RANDOM() LIMIT {n};""").fetchall()
    
    # close the database
    db.close()

    return ran_messages

{::options parse_block_html="true" /}
<div class="got-help">
    
I used to use the for loop to extract the n random messages for my `random_messages(n)` function, which takes more space than my current code.
    
Then, I use `fetchone()` to grab the messages but there is merely one character appearing for both user's handle and message. Then, I found the difference between `fetchall()` and `fetchone()` that `fetchall()` can fetch all the rows of a query result but `fetchone()` only returns a single record. So, using `fetchall()` when we need to catch more information.
     
</div>
{::options parse_block_html="false" /}

Use `random_messages(n)` function to print n messages, here I choose n=5, which the view pages will print 5 messages corresponding to the users.

In [ ]:
@app.route('/view/')
def view():

    ran_messages = random_messages(5)

    return render_template('view.html', messages = ran_messages)

![blog3_view.png](/images/blog3_view.png)

# §4. Build Webpage

Cool! we can combine and organize all functions in the `app.py` above and put all `.html` templates to `template` file.

Do not forget to import the library we need to use.

In [ ]:
from flask import Flask, g, render_template, request
import sqlite3


app = Flask(__name__)

@app.route('/')
def main():

    return render_template('main.html')


@app.route('/submit/', methods=['POST', 'GET'])
def submit():

    if request.method == 'GET':
        return render_template('submit.html')
    else:
        try:
            insert_message(request)
            return render_template('submit.html', thanks = True)
        except:
            return render_template('submit.html', error = True)


@app.route('/view/')
def view():

    ran_messages = random_messages(5)

    return render_template('view.html', messages = ran_messages)



def get_message_db():

    if 'message_db' not in g:
        g.message_db = sqlite3.connect('messages_db.sqlite')
    
    cursor = g.message_db.cursor()

    cursor.execute('CREATE TABLE IF NOT EXISTS messages(id INT, handle TEXT, message TEXT);')

    return g.message_db


def insert_message(request):

    db = get_message_db()
    cursor = db.cursor()

    message = request.form["message"]
    handle = request.form["handle"]

    row_num = cursor.execute("SELECT COUNT(*) FROM messages;").fetchone()[0]
    cursor.execute(f"""INSERT INTO messages (id, handle, message) VALUES ({row_num + 1}, "{handle}", "{message}");""")
    db.commit()

    db.close()


def random_messages(n):

    db = get_message_db()
    cursor = db.cursor()

    row_num = cursor.execute("SELECT COUNT(*) FROM messages;").fetchone()[0]

    if n > row_num:
        n = row_num

    ran_messages = cursor.execute(f"""SELECT handle, message FROM messages ORDER BY RANDOM() LIMIT {n};""").fetchall()

    db.close()

    return ran_messages

# §5. Customize Your App

Back to our `base.html`, you can see there is a navigation in line 5:
```python
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
```
It directs to the `style.css` on the `static` file. And, we can change the style of our webapp by adjusting the content of `style.css`.

For more details, you can go through my `style.css` in my GitHub repository link that I attach in the end.

{::options parse_block_html="true" /}
<div class="got-help">
    
When I made changes to the `style.css` on my site, and I refreshed the site, none of the changes were there.
Then, I found the solution by googling that I can update my web by using this: 
    
`Cmd` + `Shift` + `R`

This will force my browser to reload and refresh all the resources related to the website page.

So every time I change something in your `style.css` and I wanna view the new results immediately, use this.
    
The solution link is :
    
https://stackoverflow.com/questions/12717993/stylesheet-not-updating-when-i-refresh-my-site
    
</div>
{::options parse_block_html="false" /}

# §6. Open our Webapp

Finally, we need to run these lines on terminal and then obtain our serve address of our webapp:

**export FLASK_ENV=development; flask run**

Congratulations! We're done with our webapp.

Here’s the link to my GitHub repository containing the code for the app.:

https://github.com/FelixLQJiang/Blog/tree/main/blog3/app